# Use SPARQL endpoints to retrieve cultural data
- testrun @ 2023-06-16 and added code on 2023-07-03, 2024-05-20 through 2024-06-03
- SPARQL (“SPARQL Protocol And RDF Query Language”) is a W3C standard for querying RDF and can be used to express queries across diverse data sources, whether the data is stored natively as RDF or viewed as RDF via middleware
- SPARQLWrapper is a simple Python wrapper around a SPARQL service for remote query execution. Not only does it enable us to write more complex queries to extract information from RDF than those exposed through a library like rdflib, it can also convert query results into other formats like JSON and CSV!

## Literature
- https://rebeccabilbro.github.io/sparql-from-python/
- https://groups.google.com/g/gettyvocablod/c/mSnqx3rd8lM/m/LKPstWJyAwAJ
- https://sparqlwrapper.readthedocs.io/en/stable/main.html
- https://github.com/RDFLib/sparqlwrapper/blob/master/scripts/example.py


## CSV example
```sparql.setReturnFormat(CSV)
results = sparql.query().convert()
print(results)'''```

# Import

## Import libraries

In [ ]:
# creating time stamps
import time

# save and dump
import pickle

# data wranling
import pandas as pd

# use sparqlwrapper to access getty endpoints
from SPARQLWrapper import SPARQLWrapper, JSON, CSV

# surpress warnings, due to my code block of exploding and comparing lists
import warnings
warnings.filterwarnings('ignore')

# Tests

## Test | SPARQLWrapper on wikipedia data

In [ ]:
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

# Below we SELECT both the hot sauce items & their labels
# in the WHERE clause we specify that we want labels as well as items
sparql.setQuery("""
SELECT ?item ?itemLabel

WHERE {
  ?item wdt:P279 wd:Q522171.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
""")
# sparql.setReturnFormat(CSV)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

results_df = pd.json_normalize(results['results']['bindings'])

results_df.head()

## Test | SPARQLWrapper on ULAN endpoint, works
- retrieves uri for all Person, Artist (ULAN facet).

In [ ]:
%%time

sparql = SPARQLWrapper("http://vocab.getty.edu/sparql")

# Below we SELECT both the hot sauce items & their labels
# in the WHERE clause we specify that we want labels as well as items
sparql.setQuery("""
SELECT * WHERE { ulan:500000002 skos:member ?p . }
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

In [ ]:
results_df = pd.json_normalize(results['results']['bindings'])
results_df

# Getty SPARQL queries via SPARQLWrapper, all queries are successful

## ULAN | Full query, returns all subjects within ULAN
- https://www.getty.edu/vow/ULANFullDisplay?find=&role=&nation=&page=1&subjectid=500000002

**Persons, Artists (ULAN facet)** Note: Records under this level represent information for individuals involved in the creation or production of works of fine art or architecture, for example painters, sculptors, printmakers, and architects. Included are individuals whose biographies are well known (e.g., Rembrandt van Rijn (Dutch painter and printmaker, 1606-1669)) as well as anonymous creators with identified oeuvres but whose names are unknown and whose biography is surmised (e.g., Master of Alkmaar (North Netherlandish painter, active ca. 1490-ca. 1510)). Craftsmen, artisans, engineers, and others who create visual works are included here, even if their works are not considered fine art per se. People whose primary life roles were other than "artist" or "architect," but who created or designed art or architecture in a professional or amateur capacity, are included here with a non-preferred relationship to this facet (e.g., Thomas Jefferson (American statesman, architect, and draftsman, 1743-1826)). Performance artists are included here. 

### New SPARQL-query

In [ ]:
%%time

# set sparql endpoint
sparql = SPARQLWrapper("http://vocab.getty.edu/sparql")

# query
sparql.setQuery("""
select ?x ?name ?bio ?nationality ?type ?ScopeNote 
{
  ?x gvp:broaderExtended ulan:500000002. # Persons, Artists
  optional {?x gvp:agentTypePreferred [gvp:prefLabelGVP [xl:literalForm ?type]]}
  optional {?x foaf:focus [gvp:nationalityPreferred [gvp:prefLabelGVP [xl:literalForm ?nationality]]]}
  optional {?x gvp:prefLabelGVP [xl:literalForm ?name]}
  optional {?x foaf:focus [gvp:biographyPreferred [schema:description ?bio]]}
  optional {?x foaf:focus/gvp:biographyPreferred [gvp:estStart ?birth].}
  optional {?x skos:scopeNote [dct:language gvp_lang:en; rdf:value ?ScopeNote]}}
  }
""")

# returns results as a json
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

# traverses the json
results_df = pd.json_normalize(results['results']['bindings'])
results_df.head()

In [ ]:
with open('data_dumps/results_df_ulan_all.pickle', 'wb') as handle:
    pickle.dump(results_df, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# with open('data_dumps/results_df_all_ulan.pickle', 'rb') as handle:
#     df = pickle.load(handle)

In [ ]:
results_df.shape

In [ ]:
%store results_df

### Initial query

In [ ]:
%%time

# set sparql endpoint
sparql = SPARQLWrapper("http://vocab.getty.edu/sparql")

# query
sparql.setQuery("""
PREFIX tgn: <http://vocab.getty.edu/tgn/>
PREFIX gvp: <http://vocab.getty.edu/ontology#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX bibo: <http://purl.org/ontology/bibo/>
PREFIX skosxl: <http://www.w3.org/2008/05/skos-xl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX iso: <http://purl.org/iso25964/skos-thes#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX schema: <http://schema.org/>
PREFIX aat: <http://vocab.getty.edu/aat/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>

select ?p ?name ?bio ?birth ?death ?ScopeNote ?related ?rname ?rbirth ?rdeath ?relatedScopeNote
{ ulan:500000002 skos:member ?p .
optional {?p gvp:prefLabelGVP/xl:literalForm ?name;
     	foaf:focus/gvp:biographyPreferred [
       	schema:description ?bio;
       	gvp:estStart ?birth].}
optional { ?p gvp:prefLabelGVP/xl:literalForm ?name;
     	foaf:focus/gvp:biographyPreferred [
       	schema:description ?bio;
       	gvp:estEnd ?death]. }
optional {?p skos:related ?related . 
         	?related skos:scopeNote [dct:language gvp_lang:en; 
rdf:value ?relatedScopeNote]}  
optional {?related gvp:prefLabelGVP/xl:literalForm ?rname;
 	foaf:focus/gvp:biographyPreferred [
       	schema:description ?bio;
       	gvp:estStart ?rbirth].}
optional { ?related gvp:prefLabelGVP/xl:literalForm ?rname;
     	foaf:focus/gvp:biographyPreferred [
       	schema:description ?bio;
       	gvp:estEnd ?rdeath]. }
optional {?p skos:scopeNote [dct:language gvp_lang:en; rdf:value ?ScopeNote]}}
""")

# returns results as a json
sparql.setReturnFormat(JSON)

results = sparql.query().convert()

In [ ]:
# traverses the json
results_df = pd.json_normalize(results['results']['bindings'])
results_df.head()

### Load and save file as a pickle

In [ ]:
# with open('result_json.pickle', 'wb') as handle:
#     pickle.dump(results, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('result_json.pickle', 'rb') as handle:
#     b = pickle.load(handle)

# with open('results_df_all_ulan.pickle', 'wb') as handle:
#     pickle.dump(results_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('data_dumps/results_df_all_ulan.pickle', 'rb') as handle:
    df = pickle.load(handle)

In [ ]:
df.shape

In [ ]:
df.head()

### ULAN | AL persons and artists

In [ ]:
%%time

# set sparql endpoint
sparql = SPARQLWrapper("http://vocab.getty.edu/sparql")

# query
sparql.setQuery("""
PREFIX tgn: <http://vocab.getty.edu/tgn/>
PREFIX gvp: <http://vocab.getty.edu/ontology#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX bibo: <http://purl.org/ontology/bibo/>
PREFIX skosxl: <http://www.w3.org/2008/05/skos-xl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX iso: <http://purl.org/iso25964/skos-thes#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX schema: <http://schema.org/>
PREFIX aat: <http://vocab.getty.edu/aat/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>

SELECT ?ulan_id ?name ?bio ?birth ?death ?scopenote {
{SELECT DISTINCT ?ulan_id
         {?ulan_id foaf:focus/bio:event/(schema:location|(schema:location/gvp:broaderExtended)) ?x}}
OPTIONAL { ?ulan_id gvp:prefLabelGVP/xl:literalForm ?name;
          foaf:focus/gvp:biographyPreferred [
          schema:description ?bio;
          gvp:estStart ?birth] . }
OPTIONAL { ?ulan_id gvp:prefLabelGVP/xl:literalForm ?name;
          foaf:focus/gvp:biographyPreferred [
		  schema:description ?bio;
          gvp:estEnd ?death] . }
optional {?ulan_id skos:scopeNote [dct:language gvp_lang:en; rdf:value ?scopenote]} 
""")
#  FILTER ("1400"^^xsd:gYear <= ?birth && ?birth <= "1800"^^xsd:gYear)} # removed filter

# returns results as a json
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

# traverses the json
results_df = pd.json_normalize(results['results']['bindings'])
results_df.shape

## ULAN | Roman active painters between 1400 and 1800
**Rome (inhabited place)** 
Note: City positioned on 7 hills over the swampy Tiber river area; one of the oldest continuously occupied sites in Europe. Archaeological evidence attests to human occupation of the area from ca. 14,000 years ago, but the dense layer of later debris obscures Palaeolithic and Neolithic sites. Was an Etruscan city by 8th cen. BCE, their kings expelled and republic established by 500 BCE; soon ruled vast area and was center of Empire from 31 BCE; declined when capital moved to Constantinople in 330 CE; revived under popes.

In [ ]:
# set sparql endpoint
sparql = SPARQLWrapper("http://vocab.getty.edu/sparql")

# query
sparql.setQuery("""
PREFIX tgn: <http://vocab.getty.edu/tgn/>
PREFIX gvp: <http://vocab.getty.edu/ontology#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX bibo: <http://purl.org/ontology/bibo/>
PREFIX skosxl: <http://www.w3.org/2008/05/skos-xl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX iso: <http://purl.org/iso25964/skos-thes#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX schema: <http://schema.org/>
PREFIX aat: <http://vocab.getty.edu/aat/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>


SELECT ?id ?name ?bio ?birth ?death {
{SELECT DISTINCT ?id
         {?id foaf:focus/bio:event/(schema:location|(schema:location/gvp:broaderExtended)) tgn:7000874-place}}
OPTIONAL { ?id gvp:prefLabelGVP/xl:literalForm ?name;
          foaf:focus/gvp:biographyPreferred [
          schema:description ?bio;
          gvp:estStart ?birth] . }
OPTIONAL { ?id gvp:prefLabelGVP/xl:literalForm ?name;
          foaf:focus/gvp:biographyPreferred [
		  schema:description ?bio;
          gvp:estEnd ?death] . }
FILTER ("1400"^^xsd:gYear <= ?birth && ?birth <= "1800"^^xsd:gYear)
}

""")

# returns results as a json
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

# traverses the json
results_df = pd.json_normalize(results['results']['bindings'])
results_df.shape

In [ ]:
# dumps df as a pickle file
with open('results_df_roman.pickle', 'wb') as handle:
    pickle.dump(results_df, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# # reads pickle file
# with open('results_df_roman.pickle', 'rb') as handle:
#     df = pickle.load(handle)

### Subselects data 

In [ ]:
# subset cols
df = df[['related.value', 'rname.value', 'rbirth.value', 'rdeath.value']]

# change datatype
df['rbirth.value'] = df['rbirth.value'].astype(float)
df['rdeath.value'] = df['rdeath.value'].astype(float)

# get rid of floats, checked can be deleted
df = df[(df['rdeath.value'].notnull()) &
        (df['rbirth.value'].notnull())]

# subselection on active painters
df = df[(df['rdeath.value'].astype(int) < 1775) &
        (df['rbirth.value'].astype(int) > 1400)]

# split data and names
df[['last_name','first_name','addition','comment']] = df['rname.value'].str.split(', ', expand=True)

## ULAN | European active painters between 1400 and 1800
**Europe (continent)** Note: Europe was an early home to Homo erectus, Neanderthal and modern Homo sapiens. Agricultural settlements arose in the 7th millennium BCE. Advanced civilizations developed here in the Mediterranean area with Asian and African influences. Modern European nations began to form after the fall of the Western Roman Empire in the fifth century CE. The late 20th and early 21st centuries have witnessed the formation and expansion of the European Union, a group of European nations allied for trade and some administrative purposes, such as a common currency. 

In [ ]:
# set sparql endpoint
sparql = SPARQLWrapper("http://vocab.getty.edu/sparql")

# query
sparql.setQuery("""
PREFIX tgn: <http://vocab.getty.edu/tgn/>
PREFIX gvp: <http://vocab.getty.edu/ontology#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX bibo: <http://purl.org/ontology/bibo/>
PREFIX skosxl: <http://www.w3.org/2008/05/skos-xl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX iso: <http://purl.org/iso25964/skos-thes#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX schema: <http://schema.org/>
PREFIX aat: <http://vocab.getty.edu/aat/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>


SELECT ?id ?name ?bio ?birth ?death {
{SELECT DISTINCT ?id
         {?id foaf:focus/bio:event/(schema:location|(schema:location/gvp:broaderExtended)) tgn:7000874-place}}
OPTIONAL { ?id gvp:prefLabelGVP/xl:literalForm ?name;
          foaf:focus/gvp:biographyPreferred [
          schema:description ?bio;
          gvp:estStart ?birth] . }
OPTIONAL { ?id gvp:prefLabelGVP/xl:literalForm ?name;
          foaf:focus/gvp:biographyPreferred [
		  schema:description ?bio;
          gvp:estEnd ?death] . }
FILTER ("1400"^^xsd:gYear <= ?birth && ?birth <= "1800"^^xsd:gYear)}

""")

# returns results as a json
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

# traverses the json
results_df = pd.json_normalize(results['results']['bindings'])
results_df.shape

In [ ]:
with open('data_dumps/results_df_roman_painters_1400_1800.pickle', 'wb') as handle:
    pickle.dump(results_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

## ULAN | Active between 1400 and 1750

In [ ]:
# set sparql endpoint
sparql = SPARQLWrapper("http://vocab.getty.edu/sparql")

# query
sparql.setQuery("""
PREFIX tgn: <http://vocab.getty.edu/tgn/>
PREFIX gvp: <http://vocab.getty.edu/ontology#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX bibo: <http://purl.org/ontology/bibo/>
PREFIX skosxl: <http://www.w3.org/2008/05/skos-xl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX iso: <http://purl.org/iso25964/skos-thes#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX schema: <http://schema.org/>
PREFIX aat: <http://vocab.getty.edu/aat/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>


SELECT ?id ?name ?bio ?place ?birth ?death {
{SELECT DISTINCT ?id
         {?id foaf:focus/bio:event/(schema:location|(schema:location/gvp:broaderExtended)) ?place}}
OPTIONAL { ?id gvp:prefLabelGVP/xl:literalForm ?name;
          foaf:focus/gvp:biographyPreferred [
          schema:description ?bio;
          gvp:estStart ?birth] . }
OPTIONAL { ?id gvp:prefLabelGVP/xl:literalForm ?name;
          foaf:focus/gvp:biographyPreferred [
		  schema:description ?bio;
          gvp:estEnd ?death] . }
FILTER ("1400"^^xsd:gYear <= ?birth && ?birth <= "1750"^^xsd:gYear)}
""")

# returns results as a json
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

# traverses the json
results_df = pd.json_normalize(results['results']['bindings'])
results_df

## ULAN | All painters and artists, birth and death dates, nationalities, scopenote, etc.

In [ ]:
%%time

# set sparql endpoint
sparql = SPARQLWrapper("http://vocab.getty.edu/sparql")

# query
sparql.setQuery("""
PREFIX tgn: <http://vocab.getty.edu/tgn/>
PREFIX gvp: <http://vocab.getty.edu/ontology#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX bibo: <http://purl.org/ontology/bibo/>
PREFIX skosxl: <http://www.w3.org/2008/05/skos-xl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX iso: <http://purl.org/iso25964/skos-thes#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX schema: <http://schema.org/>
PREFIX aat: <http://vocab.getty.edu/aat/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>

select ?ulan_id ?name ?type ?bio ?birth ?death ?nationality ?scopenote 
{
  ?ulan_id gvp:broaderExtended ulan:500000002. # Persons, Artists
  optional {?ulan_id gvp:prefLabelGVP [xl:literalForm ?name]}  
  optional {?ulan_id gvp:agentTypePreferred [gvp:prefLabelGVP [xl:literalForm ?type]]}
  optional {?ulan_id foaf:focus [gvp:biographyPreferred [schema:description ?bio]]}
  optional {?ulan_id foaf:focus/gvp:biographyPreferred [gvp:estStart ?birth].}
  optional {?ulan_id foaf:focus/gvp:biographyPreferred [gvp:estEnd ?death]. }
  optional {?ulan_id foaf:focus [gvp:nationalityPreferred [gvp:prefLabelGVP [xl:literalForm ?nationality]]]}
  optional {?ulan_id skos:scopeNote [dct:language gvp_lang:en; rdf:value ?scopenote]}} 
  """)

print(f"query done.")

# returns results as a json
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

print(f"results converted.")

# traverses the json
results_df = pd.json_normalize(results['results']['bindings'])
results_df.shape

In [ ]:
results_df.shape

In [ ]:
results_df = (results_df[[
            # 'ulan_id.type', 
            'ulan_id.value', 
            # 'name.xml:lang', 
            # 'name.type',
            'name.value', 
            # 'type.xml:lang', 
            # 'type.type', 
            'type.value', 
            # 'bio.type',
            'bio.value', 
            # 'birth.datatype',
            # 'birth.type',
            'birth.value',
            'death.datatype',
            # 'death.type', 
            'death.value', 
            # 'nationality.xml:lang',
            # 'nationality.type', 
            'nationality.value',
            # 'scopenote.xml:lang',
            # 'scopenote.type', 
            'scopenote.value'
            ]].drop_duplicates())

In [ ]:
results_df.head()

In [ ]:
time_stamp = time.strftime('%Y%m%d-%H%M%S')
with open(f'data_dumps/{time_stamp}_df_ulan.pickle', 'wb') as handle:
    pickle.dump(results_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

## AAT | Associated Concepts Facet
The AAT includes **generic terms, and associated dates, relationships, and other information about concepts related to or required to catalog, discover, and retrieve information about art, architecture, and other visual cultural heritage**, including related disciplines dealing with visual works, such as archaeology and conservation, where the works are of the type collected by art museums and repositories for visual cultural heritage, or that are architecture. It is our goal to be ever more inclusive of various cultures and their visual works. Also, in recognition of diverse collections found in art museums, the AAT contains terminology to describe objects and associated activities that are ceremonial or utilitarian in nature, but are not necessarily labeled as art according to traditional Western aesthetics. 

In [ ]:
%%time
# set sparql endpoint
sparql = SPARQLWrapper("http://vocab.getty.edu/sparql")

# query
sparql.setQuery("""
PREFIX tgn: <http://vocab.getty.edu/tgn/>
PREFIX gvp: <http://vocab.getty.edu/ontology#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX bibo: <http://purl.org/ontology/bibo/>
PREFIX skosxl: <http://www.w3.org/2008/05/skos-xl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX iso: <http://purl.org/iso25964/skos-thes#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX schema: <http://schema.org/>
PREFIX aat: <http://vocab.getty.edu/aat/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>

select distinct ?Subject ?ScopeNote ?Term
{
  ?Subject gvp:broaderExtended aat:300264086. # Associated Concepts Facet
   optional {?Subject skos:scopeNote [dct:language gvp_lang:en; rdf:value ?ScopeNote]}
   optional {?Subject gvp:prefLabelGVP [xl:literalForm ?Term]}
   }
""")

# returns results as a json
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

# traverses the json
results_df = pd.json_normalize(results['results']['bindings'])
results_df.head()

In [ ]:
results_df.shape

In [ ]:
%store results_df

In [ ]:
with open('data_dumps/results_df_aat_300264086.pickle', 'wb') as handle:
    pickle.dump(results_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

## TGN | All places, their names, long- and latitude, regions, continents, etc.
 A minimum **TGN record** contains a numeric ID, a name, a place in the hierarchy, and a place type. However, the data model includes dates, relationships, and other rich data. Most records include coordinates. TGN focuses on the historical world and places necessary for cataloging and discovery of visual works. It is not intended to be comprehensive. Current areas of TGN development include 1) adding archaeological sites, lost sites, and other historical sites, particularly Pre-Columbian places and places in Asia, Middle East, Africa, and others, and 2) building hierarchies for historical nations and empires. 

In [ ]:
%%time
# set sparql endpoint
sparql = SPARQLWrapper("http://vocab.getty.edu/sparql")

# query
sparql.setQuery("""
select ?tgn_id ?city_name ?lat ?long ?parentstring ?broader_parentstring {{
  ?tgn_id skos:inScheme tgn: ; }
OPTIONAL { ?tgn_id foaf:focus [wgs:lat ?lat; wgs:long ?long];
    gvp:prefLabelGVP [xl:literalForm ?city_name].}
OPTIONAL { ?tgn_id gvp:parentString ?parentstring. }
OPTIONAL { ?tgn_id gvp:broaderPartitive ?x .
           ?x gvp:parentString ?broader_parentstring}
} 
""")

print(f"query done.")

# returns results as a json
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

print(f"results converted.")

# traverses the json
results_df = pd.json_normalize(results['results']['bindings'])
results_df.shape

In [ ]:
# delete empty results
results_df = results_df[(results_df['city_name.value'].notnull())]

# subset results and rename cols
results_df = results_df[[
#     'tgn_id.type', 
    'tgn_id.value', 
    'city_name.value',
#     'parentstring.type', 
    'parentstring.value',
#     'broader_parentstring.type',
    'broader_parentstring.value',
#     'city_name.xml:lang', 
#     'city_name.type', 
#     'lat.datatype', 
#     'lat.type',       
    'lat.value', 
#     'long.datatype', 
#     'long.type', 
    'long.value'
           ]].rename(columns={'tgn_id.value' : 'tgn_id', 
                              'city_name.value' : 'city_name', 
                              'parentstring.value' : 'parentstring',
                              'broader_parentstring.value' : 'broader_parentstring',
                              'lat.value' : 'lat',
                              'long.value' : 'lon',
                             })

results_df.shape

In [ ]:
%%time
# compare two cols with strings, converted to list

# initiate col
results_df['inferred_city_name'] = ''

# iterate over df
for i in range(len(tqdm.notebook.tqdm(results_df, total=len(results_df)))):
    
    # check for empty values
    if type(results_df['parentstring'].iloc[i]) != float and type(results_df['broader_parentstring'].iloc[i]) != float:
        
        # split and create two lists to compare
        test = results_df['parentstring'].iloc[i].split(', ')
        test2 = results_df['broader_parentstring'].iloc[i].split(', ')
        
    # iterate over list and return diff
        for item in test:
            if item not in test2:
                results_df['inferred_city_name'].iloc[i] = item
    else:
        results_df['inferred_city_name'].iloc[i] = ''

In [ ]:
# reorder cols
results_df = results_df[['tgn_id', 'city_name', 'inferred_city_name', 'parentstring', 'broader_parentstring', 'lat',
       'lon']]

In [ ]:
results_df.head()

In [ ]:
with open('data_dumps/results_df_tgn_country.pickle', 'wb') as handle:
    pickle.dump(results_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

# RKD

## RKDartists | all subjects with a name
In RKDartists, data are maintained on more than **350,000 Dutch and Flemish artists**, art dealers, collectors and other art-related figures. Biographical data such as birth and death dates, family relationships and workplaces of artists are extracted from RKDartists.

In [ ]:
%%time
# set sparql endpoint
sparql = SPARQLWrapper("https://api.data.netwerkdigitaalerfgoed.nl/datasets/rkd/rkdartists/services/rkdartists/sparql")

# query
sparql.setQuery("""
PREFIX rkd_birth: <http://data.rkd.nl/def#Birth>
PREFIX rkd_death: <http://data.rkd.nl/def#Death>
PREFIX schema: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT ?rkd_link ?name ?altname ?actor ?rkd_id ?startdate ?enddate ?description 
WHERE {
    ?rkd_link schema:name ?name . 
    OPTIONAL {?rkd_link schema:description ?description}
    OPTIONAL { ?rkd_link schema:actor ?actor.}
    OPTIONAL { ?rkd_link schema:alternateName ?altname. }
    OPTIONAL { ?rkd_link schema:identifier ?rkd_id . }
    OPTIONAL { ?rkd_link rkd_birth: ?birth . }
    OPTIONAL { ?birth schema:endDate ?startdate . }
    OPTIONAL { ?rkd_link rkd_death: ?death . }
    OPTIONAL { ?death schema:startDate ?enddate . }
    } 
""")

print(f"query done.")

# returns results as a json
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

print(f"results converted.")

# traverses the json
results_df = pd.json_normalize(results['results']['bindings'])
results_df.shape

In [ ]:
# delete empty results
results_df = results_df[(results_df['rkd_link.value'].notnull())]

# subset results and rename cols
results_df = (results_df[[
    # 'rkd_link.type', 
    'rkd_link.value',
    # 'name.type',
    'name.value',
    # 'name.xml:lang',
    # 'altname.type',
    'altname.value',
    # 'rkd_id.type',
    'rkd_id.value',
    # 'startdate.type',
    'startdate.value',
    # 'startdate.datatype',
    # 'enddate.type',
    'enddate.value',
    # 'enddate.datatype',
    # 'description.type',
    'description.value',
    # 'description.xml:lang',
    # 'rkd_id.datatype'
    ]])

# rename cols
for col in results_df.columns:
    results_df.rename(columns={col:col.replace('.value', '')}, inplace=True)
    
# splits description col
results_df['description_prefix'] = results_df['description'].str.split(':').str[0]
results_df['description'] = results_df['description'].str.split(':').str[1]

# reorder cols
results_df = results_df[['rkd_link', 'name', 'altname', 'rkd_id', 'startdate', 'enddate','description_prefix', 'description']]
results_df.head()

In [ ]:
with open('data_dumps/results_df_rkd_first_10000.pickle', 'wb') as handle:
    pickle.dump(results_df, handle, protocol=pickle.HIGHEST_PROTOCOL)